<a href="https://colab.research.google.com/github/GavinButts/Fall-Data-Challenge-2022/blob/main/Code/Kevin/Ordinal%20-%20SEABSNT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns
!pip install --upgrade --no-deps statsmodels
import scipy.stats as stats


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 4.4 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


In [2]:
sheet_id = "11rno0SjsOyV1JlevH3q40UMm1s5e6m7g"
sheet_name = "curated_2019-required"

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df = pd.read_csv(url)


In [3]:
#create FOYESCNT variable that determines the amount of times each person said 'Yes' on the FO#### questions (ITEMS 72-74)
#Legend:  Number of 'Yes'

FOYESCNT = []      #variable that will be added to csv
yesCount = 0
for i in range(15500):
  if df['FOSTORY2X'][i] == 1:
    yesCount += 1
  if df['FOCRAFTS'][i] == 1:
    yesCount += 1
  if df['FOGAMES'][i] == 1:
    yesCount += 1
  if df['FOBUILDX'][i] == 1:
    yesCount += 1
  if df['FOSPORT'][i] == 1:
    yesCount += 1
  if df['FORESPON'][i] == 1:
    yesCount += 1
  if df['FOHISTX'][i] == 1:
    yesCount += 1
  if df['FOLIBRAYX'][i] == 1:
    yesCount += 1
  if df['FOBOOKSTX'][i] == 1:
    yesCount += 1
  if df['FODINNERX'][i] != 0:
    yesCount += 1
  FOYESCNT.append(yesCount)
  yesCount = 0

df["FOYESCNT"] = FOYESCNT
df.to_csv("sample.csv", index=False)  #adds variable to csv

#create FSYESCNT variable that determines the amount of times each person said 'Yes' on the FS#### questions (ITEMS 60)
#Legend:  Number of 'Yes'

FSYESCNT = []      #variable that will be added to csv
yesCount = 0
FSYESCNT_LMH = []

for i in range(15500):
  if df['FSSPORTX'][i] == 1:
    yesCount += 1
  if df['FSVOL'][i] == 1:
    yesCount += 1
  if df['FSMTNG'][i] == 1:
    yesCount += 1
  if df['FSPTMTNG'][i] == 1:
    yesCount += 1
  if df['FSATCNFN'][i] == 1:
    yesCount += 1
  if df['FSFUNDRS'][i] == 1:
    yesCount += 1
  if df['FSCOMMTE'][i] == 1:
    yesCount += 1
  if df['FSCOUNSLR'][i] == 1:
    yesCount += 1
  FSYESCNT.append(yesCount)
  if yesCount <= 3:
    FSYESCNT_LMH.append('1) LOW INVOLVMENT')
  elif yesCount > 3 and yesCount<=7:
    FSYESCNT_LMH.append('2) MEDIUM INVOLVEMENT')
  elif yesCount > 7:
    FSYESCNT_LMH.append('3) HIGH INVOLVEMENT')
  yesCount = 0

df["FSYESCNT"] = FSYESCNT
df["FSYESCNT_LMH"] = FSYESCNT_LMH

df.to_csv("sample.csv", index=False)  #adds variable to csv


In [4]:
#Data Pre-Processing
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=[-1,1,2,3,4], ordered=True)
df["SEENJOY"] = df["SEENJOY"].astype(cat_type)
df["SEENJOY"].dtype

CategoricalDtype(categories=[-1, 1, 2, 3, 4], ordered=True)

In [5]:

from statsmodels.miscmodels.ordinal_model import OrderedModel
df

,BASMID,ALLGRADEX,EDCPUB,SCCHOICE,SPUBCHOIX,SCONSIDR,SCHLHRSWK,EINTNET,MOSTIMPT,INTNUM,...,HHPARN19_BRD,NUMSIBSX,PARGRADEX,RACEETH,INTACC,CENREG,ZIPLOCL,FOYESCNT,FSYESCNT,FSYESCNT_LMH
0,20191000012,12,1,2,3,2,4,4,-1,-1,...,1,1,3,4,1,4,11,6,3,1) LOW INVOLVMENT
1,20191000029,4,1,2,3,2,4,3,-1,2,...,2,1,3,3,1,4,12,4,1,1) LOW INVOLVMENT
2,20191000059,11,2,2,2,1,4,3,2,2,...,2,1,5,2,3,2,11,6,7,2) MEDIUM INVOLVEMENT
3,20191000070,6,1,1,1,1,4,4,-1,-1,...,2,1,2,3,1,2,31,10,8,3) HIGH INVOLVEMENT
4,20191000078,9,1,2,3,2,4,4,-1,-1,...,1,1,5,5,1,2,21,6,4,2) MEDIUM INVOLVEMENT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15495,20191225472,7,1,2,2,2,4,4,-1,-1,...,1,2,4,4,1,3,23,5,4,2) MEDIUM INVOLVEMENT
15496,20191225475,2,1,2,2,2,4,4,-1,-1,...,1,3,5,2,1,2,22,7,3,1) LOW INVOLVMENT
15497,20191225477,12,1,1,3,2,4,4,-1,-1,...,1,2,2,3,1,1,13,7,5,2) MEDIUM INVOLVEMENT
15498,20191225479,13,1,2,2,1,4,4,-1,-1,...,2,0,4,1,3,4,11,8,7,2) MEDIUM INVOLVEMENT


In [119]:
#['HHPARN19X','FHPLACE','SEABSNT','FSYESCNT','SEGRADES','FOYESCNT','RACEETH','TTLHHINC']
x = ['ALLGRADEX','EDCPUB','SCCHOICE','SCONSIDR','SEGRADEQ','FCSCHOOL','FCTEACHR','FCORDER','FCSUPPRT','FHPLACE','HDHEALTH','P1SEX',
     'TTLHHINC','SEABSNT','FSYESCNT','FOYESCNT','RACEETH','OWNRNTHB','SEFUTUREX','PARGRADEX']
mod_prob = OrderedModel(df['SEENJOY'],
                        df[x],
                        distr='logit')
 
res_log = mod_prob.fit(method='bfgs')
res_log.summary()


Optimization terminated successfully.
         Current function value: 0.831102
         Iterations: 88
         Function evaluations: 90
         Gradient evaluations: 90


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                SEENJOY   Log-Likelihood:                -12882.
Model:                   OrderedModel   AIC:                         2.581e+04
Method:            Maximum Likelihood   BIC:                         2.601e+04
Date:                Fri, 28 Oct 2022                                         
Time:                        01:08:02                                         
No. Observations:               15500                                         
Df Residuals:                   15475                                         
Df Model:                          25                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ALLGRADEX      0.0881      0.005     18.097      0.000       0.079       0.098
EDCPUB        -0.2309      0.056     -4.091      0.000      -0.342      -0.120
SCCHOICE       0.1805      0.036      4.963      0.000       0.109       0.252
SCONSIDR      -0.1907      0.036     -5.289      0.000      -0.261      -0.120
SEGRADEQ       0.6613      0.022     30.072      0.000       0.618       0.704
FCSCHOOL       0.5557      0.041     13.572      0.000       0.475       0.636
FCTEACHR       0.2283      0.036      6.383      0.000       0.158       0.298
FCORDER        0.2312      0.032      7.149      0.000       0.168       0.295
FCSUPPRT       0.0933      0.032      2.893      0.004       0.030       0.156
FHPLACE        0.0750      0.026      2.909      0.004       0.024       0.125
HDHEALTH       0.2032      0.024      8.638      0.000       0.157       0.249
P1SEX          0.1074      0.036      2.993      0.003       0.037       0.178
TTLHHINC       0.0319      0.007      4.361      0.000       0.018       0.046
SEABSNT        0.3100      0.031      9.891      0.000       0.249       0.371
FSYESCNT      -0.0468      0.009     -4.945      0.000      -0.065      -0.028
FOYESCNT      -0.0837      0.009     -9.821      0.000      -0.100      -0.067
RACEETH       -0.0883      0.013     -6.581      0.000      -0.115      -0.062
OWNRNTHB      -0.1249      0.039     -3.177      0.001      -0.202      -0.048
SEFUTUREX     -0.1325      0.016     -8.064      0.000      -0.165      -0.100
PARGRADEX      0.0778      0.019      4.199      0.000       0.041       0.114
NUMSIBSX       0.0140      0.017      0.805      0.421      -0.020       0.048
-1/1          -6.3781      0.736     -8.665      0.000      -7.821      -4.935
1/2            2.1911      0.079     27.710      0.000       2.036       2.346
2/3            1.2442      0.011    114.204      0.000       1.223       1.266
3/4            0.7464      0.027     27.769      0.000       0.694       0.799
==============================================================================
"""

In [115]:
predicted = res_log.model.predict(res_log.params, exog=df[x])
predicted


/usr/local/lib/python3.7/dist-packages/statsmodels/miscmodels/ordinal_model.py:419: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  xb = xb[:, None]


array([[9.75438454e-05, 4.27832744e-01, 5.32131351e-01, 3.49175251e-02,
        5.02083578e-03],
       [8.98207279e-05, 4.07776129e-01, 5.48909756e-01, 3.77740600e-02,
        5.45023472e-03],
       [1.38371738e-06, 1.04975716e-02, 2.43771538e-01, 4.83323557e-01,
        2.62405949e-01],
       ...,
       [1.58890306e-04, 5.49099043e-01, 4.25841130e-01, 2.18128134e-02,
        3.08812325e-03],
       [3.45367229e-05, 2.09349488e-01, 6.85470699e-01, 9.10925104e-02,
        1.40527660e-02],
       [1.31999117e-05, 9.19015506e-02, 6.72940266e-01, 1.99192735e-01,
        3.59522484e-02]])

In [116]:
pred_choice = predicted.argmax(1)
print('Fraction of correct choice predictions')
print((np.asarray(df['SEENJOY'].values.codes) == pred_choice).mean())


Fraction of correct choice predictions
0.6357419354838709
